# openCV 作業：
* 題目：請找一段約一分鐘左右的影片，利用上課時學過的影像處理功能，發揮所有創意和想像，將該影片加工特效，產生一段新影片，怕誤解同學的創意，請將你的亮點盡可能寫在影片中的右下角

* 上傳：導師會有告訴同學上傳空間, 或可上傳到Youtube 或自己的雲端空間, 把連結給老師(依導師規範為準)

* 繳交日期：課後二週

* 評分：亮點愈亮，創意愈多，分數愈高 

* 參考 :<br>
https://www.youtube.com/watch?v=Fszu0L1JtqM

In [9]:
import numpy as np
import cv2


cap = cv2.VideoCapture('../video/grandpa.mp4')   # 讀取電腦中的影片
cap2 = cv2.VideoCapture('../video/grandpa.mp4')

fps = cap.get(cv2.CAP_PROP_FPS)               # Frame Per Second
F_Count = cap.get(cv2.CAP_PROP_FRAME_COUNT)   # frame count
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))        # 取得畫面尺寸
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'MP4V')             # 使用 XVID 編碼
out = cv2.VideoWriter('../video/granpa_out1.mp4', fourcc, fps, (w, h))

print(f'fps : {fps:0.01f} f/s, Frame_Count : {F_Count}')

for i in range(1,2936):
    ret, frame = cap.read(i) #讀取影片的每一幀
    if not ret : break
    
    c=cv2.waitKey(20) 
    if c==27 : break
    
    #原影片
    if i <= 420:
        cv2.putText(frame, 'original',(50, 130), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (255,255,255), 6, cv2.LINE_AA) 
    
    #RGB 左右相反
    if 421 <= i <= 840:
        frame = cv2.flip(frame, 1)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2RGB)
        cv2.putText(frame, 'flip(1)&cvtColor',(50, 130), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (255,255,255), 6, cv2.LINE_AA)
    
    #邊緣
    if 841 <= i <= 1260:
         #图像灰度处理
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  
        #高斯滤波降噪
        gaussian = cv2.GaussianBlur(gray, (5,5), 0)
 
        #Canny算子
        canny = cv2.Canny(gaussian, 50, 150)

        #阈值化处理
        ret, frame = cv2.threshold(canny, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        cv2.putText(frame, 'threshold',(50, 130), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0,0,0), 6, cv2.LINE_AA)

    #模糊
    if 1261 <= i <= 1680:
        frame = cv2.blur(frame,(25,25))
        cv2.putText(frame, 'blur',(50, 130), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (255,255,255), 6, cv2.LINE_AA)
    
    #母子畫面
    if 1681 <= i <= 2100:
        ret, frame2 = cap2.read(i) #讀取影片的每一幀
        if not ret : break
        frame2 = cv2.resize(frame2,(320,180))  # 縮小尺寸
        frame = cv2.resize(frame,(1280,720))  # 縮小尺寸
        frame[530:710,950:1270] = frame2       # 將 img2 的特定區域換成 img1

        cv2.rectangle(frame, (950,530), (1270,710), (255,255,0), 5)  # 繪製子影片的外框
        cv2.putText(frame, 'insert video',(50, 130), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (255,255,255), 6, cv2.LINE_AA)
    #變形
    if 2101 <= i <= 2520:
        p1 = np.float32([[200,100],[1280,0],[0,720]])
        p2 = np.float32([[0,0],[1280,0],[0,720]])
        M = cv2.getAffineTransform(p1, p2)
        frame = cv2.warpAffine(frame, M, (1280, 720))
        cv2.putText(frame, 'getAffineTransform',(50, 130), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (255,255,255), 6, cv2.LINE_AA)
    
    #馬賽克
    if 2521 <= i <= 2936:
        size = frame.shape         # 取得原始圖片的資訊
        level = 15               # 縮小比例 ( 可當作馬賽克的等級 )
        h = int(size[0]/level)   # 按照比例縮小後的高度 ( 使用 int 去除小數點 )
        w = int(size[1]/level)   # 按照比例縮小後的寬度 ( 使用 int 去除小數點 )
        frame = cv2.resize(frame, (w,h), interpolation=cv2.INTER_LINEAR)   # 根據縮小尺寸縮小
        frame = cv2.resize(frame, (size[1],size[0]), interpolation=cv2.INTER_NEAREST) 
        cv2.putText(frame, 'mosaic',(50, 130), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (255,255,255), 6, cv2.LINE_AA)
    
    cv2.putText(frame, f'{cap.get(cv2.CAP_PROP_POS_FRAMES):.0f} frames, {cap.get(cv2.CAP_PROP_POS_MSEC):.0f} ms', 
                (50 , 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255), 2, cv2.LINE_AA)
    cv2.imshow('frame',frame)
    out.write(frame)
    
out.release()
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

fps : 30.0 f/s, Frame_Count : 2935.0


-1